In [1]:
from tqdm import tqdm # Barra de progreso


import transformers
transformers.__version__

'4.19.2'

In [2]:
#!pip install protobuf==3.20.*

In [3]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "mrm8488/mbart-large-finetuned-opus-es-en-translation"#mbart-large-finetuned-bible-es-en-translation"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [4]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


### _Test con los títulos de noticias_

In [5]:
import pandas as pd
df = pd.read_csv("dataset-loslagos-8months.csv", 
                 sep=";", 
                 engine='python')
df.columns = ["date", "media_outlet", "url", "title", "text"]
print(len(df))
df.head()

29598


,date,media_outlet,url,title,text
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At..."


In [11]:
sub = df[:100].copy()
sub['title_label'] = ""
sub['title_score'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    
    # Traducimos de español a inglés
    inputs = tokenizer(row['title'], return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)
    to_eng = tokenizer.decode(outputs[0])[4:-4]
    
    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(to_eng)
    
    # Insertamos en dataframe
    sub.at[index, "title_label"] = sentiment_value[0].get('label')
    sub.at[index, "title_score"] = sentiment_value[0].get('score')


sub rows - sentiment: 100%|███████████████████████████████████████████████████████████| 100/100 [03:26<00:00,  2.07s/it]


In [14]:
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
sub[['title', "title_label", 'title_score']]

,title,title_label,title_score
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,POSITIVE,0.998974
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,POSITIVE,0.960294
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,NEGATIVE,0.965034
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,POSITIVE,0.985965
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,POSITIVE,0.999179
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,NEGATIVE,0.958992
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,NEGATIVE,0.950498
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,NEGATIVE,0.913558
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,POSITIVE,0.992521
9,CONADI llama a renovar directivas de comunidades indígenas,NEGATIVE,0.7059


### Estructura de una noticia
Según [Wikipedia](https://es.wikipedia.org/wiki/Noticia), una noticia está compuesta por las siguientes secciones:

- **Epígrafe o antetítulo**: Se sitúa al lector en el contexto sobre el que se va a hablar.
- **El título**: Se trata de resumir la información en un número limitado de palabras. El título debe despertar el interés del público, tomando en cuenta, dos elementos clave de la noticia como lo son la precisión y la concisión.
- **Bajada o subtítulo**: Aporta información adicional que resume brevemente lo sucedido.
- **Entradilla, copete o lead**: Es el primer párrafo de la noticia, en el cual se deben concentrar los datos más relevantes del hecho y/o del acontecimiento. Si el título atrae al público, el lead debe confirmar su interés.
- **El cuerpo**: Se desarrolla a profundidad lo descrito en el lead. En esta parte de la noticia, se deben ir desglosando los datos del hecho en un orden de mayor a menor importancia, siguiendo la estructura de la pirámide invertida
- **Cierre o remate**: Es el último párrafo de la nota, denominado remate, tiene por función “cerrar” la noticia; darle a entender al receptor que la noticia allí precisamente se concluye.


En base a lo anterior, se diseñará una estrategia que permita acercarnos más en la obtención del grado de polaridad de una noticia.

- <u>Estrategia 1</u>: Analizar el lead

In [35]:
sub['lead_label'] = ""
sub['lead_score'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - lead_sentiment', total=sub.shape[0]):
    
    # Obtenemos la primera oración.
    lead = row['text'].split(".")[0] 
    
    # Traducimos de español a inglés
    inputs = tokenizer(lead, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)
    to_eng = tokenizer.decode(outputs[0])[4:-4] 
    
    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(to_eng)
    
    # Insertamos en dataframe
    sub.at[index, "lead_label"] = sentiment_value[0].get('label')
    sub.at[index, "lead_score"] = sentiment_value[0].get('score')

sub rows - lead_sentiment: 100%|██████████████████████████████████████████████████████| 100/100 [05:19<00:00,  3.19s/it]


In [36]:
sub[['title', "title_label", 'title_score', "lead_label", 'lead_score']]

,title,title_label,title_score,lead_label,lead_score
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,POSITIVE,0.998974,POSITIVE,0.999579
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,POSITIVE,0.960294,POSITIVE,0.999584
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,NEGATIVE,0.965034,NEGATIVE,0.727059
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,POSITIVE,0.985965,POSITIVE,0.996008
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,POSITIVE,0.999179,POSITIVE,0.99826
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,NEGATIVE,0.958992,NEGATIVE,0.983558
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,NEGATIVE,0.950498,NEGATIVE,0.934999
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,NEGATIVE,0.913558,NEGATIVE,0.990013
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,POSITIVE,0.992521,POSITIVE,0.999629
9,CONADI llama a renovar directivas de comunidades indígenas,NEGATIVE,0.7059,POSITIVE,0.743004


__(Ahora me imagino que hay que hacer un acercamiento de este estilo, una comparación entre el sentimiento del título y el lead (?))__

In [ ]:
sub['naive_title_sentiment'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    
    label = row['title_label']
    score = float(row['title_score'])
    
    if(score > 0.9 and label=="NEGATIVE"): sub.at[index, "naive_title_sentiment"] = "NEGATIVE"
    #if(score > 0.8 and score < 0.9 and label=="NEGATIVE"): sub.at[index, "naive_title_sentiment"] = "NEUTRAL NEGATIVE"
    if(score < 0.9): sub.at[index, "naive_title_sentiment"] = "NEUTRAL"
    #if(score > 0.8 and score < 0.9 and label=="POSITIVE"): sub.at[index, "naive_title_sentiment"] = "NEUTRAL POSITIVE"
    if(score > 0.9 and label=="POSITIVE"): sub.at[index, "naive_title_sentiment"] = "POSITIVE"
        
sub.naive_title_sentiment.value_counts()
#sub.loc[sub.naive_title_sentiment == 'NEUTRAL']